In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# Fill na with the nearest data or data from the same subzone
def min_dist_feature_in_same_subzone (fill_in_feature, subzone, lat, lng, df):
    df_subset = df[df['subzone'] == subzone]
    df_subset = df_subset.reset_index(drop=True)
    return df.iloc[np.argmin(np.sqrt((df["lat"]-lat)**2+(df["lng"]-lng)**2))][fill_in_feature]

def fill_NA_with_nearest_record(df, empty_feature):
    df_empty = df[(df[empty_feature].isna())]
    df_empty[empty_feature] = df_empty.apply(lambda row: min_dist_feature_in_same_subzone(empty_feature, row['subzone'], row['lat'], row['lng'], df[(df[empty_feature].notna())]), axis=1)
    return df_empty

In [ ]:
df_train = pd.read_csv(r"train.csv")

In [ ]:
# property_type: First character uppercase -> lowercase
df_train['property_type'] = df_train['property_type'].str.lower()

# property_type: (hdb 2 rooms, hdb 3 rooms, hdb 4 rooms, hdb 5 rooms) -> hdb
temp = df_train['property_type'].str.startswith(('hdb 2', 'hdb 3', 'hdb 4', 'hdb 5'))
df_train['property_type'] = np.where((temp == True), 'hdb', df_train['property_type'])

In [ ]:
# num_beds: fill 1 with studio
df_train['num_beds'] = np.where((df_train['num_beds'].isna() & df_train['title'].str.startswith('studio ')), 
                                1, df_train['num_beds'])

In [ ]:
# price: delete rows with price value 0
df_train = df_train[df_train['price'] != 0]

# subzone & planning_area: delete rows with subzone and planning_area values NaN 
df_train = df_train[(df_train['subzone'].notna() & df_train['planning_area'].notna())]

# tenure: fill all hdb property_type with hdb defult tenure value - '99-year-leasehold'
hdb_tenure = df_train[(df_train['property_type'].str.startswith('hdb')) & df_train['tenure'].notna() ]['tenure'].unique()[0]
df_train['tenure'] = np.where((df_train['property_type'].str.startswith('hdb')) & (df_train['tenure'].isna()), hdb_tenure, df_train['tenure'])

# tenure: fill in NaN tenure with value from same property_name, otherwise drop
df_train.tenure = df_train.groupby('property_name').tenure.transform('first')

# tenure: delete rows with tenure values NaN as no useful records can be used to fill in NaN values
df_train = df_train[df_train['tenure'].notna()]

# built_year: fill in NaN built_year with value from same property_name, otherwise drop
df_train.built_year = df_train.groupby('property_name').built_year.transform('first')

df_train = df_train[df_train['built_year'].notna()]

df_train['lease_end_year'] = df_train.tenure.str.extract('(\d+)')
df_train['lease_end_year'] = np.where((df_train['tenure'] == 'freehold'), 9999, df_train['lease_end_year'])
df_train['lease_end_year'] = np.where((df_train['tenure'] == 'freehold'), 9999, df_train['lease_end_year'].astype(int) + df_train.built_year)

In [ ]:
#Delete the data with unreasonable size
from sklearn.cluster import DBSCAN

# Reset index
df_train = df_train.reset_index(drop=True)

# Remove NaN in num_beds and num_baths
df_train = df_train.dropna(subset=['num_beds'])
df_train = df_train.dropna(subset=['num_baths'])

# DBSCAN using beds to baths ratio
df_train['beds_to_baths'] = df_train['num_beds'] / df_train['num_baths']
sk_clustering_iris = DBSCAN(eps=0.5, min_samples=5).fit(df_train[['beds_to_baths']])
sk_noise_iris = np.argwhere(sk_clustering_iris.labels_ < 0).squeeze()
sk_noise_iris.sort()

for i in sk_noise_iris:
    df_train = df_train.drop(i)
    df_train = df_train.reset_index(drop=True)

    
# DBSCAN using baths to beds ratio
df_train['baths_to_beds'] = df_train['num_baths'] / df_train['num_beds']
sk_clustering_iris = DBSCAN(eps=0.5, min_samples=5).fit(df_train[['baths_to_beds']])
sk_noise_iris = np.argwhere(sk_clustering_iris.labels_ < 0).squeeze()
sk_noise_iris.sort()

for i in sk_noise_iris:
    df_train = df_train.drop(i)
    df_train = df_train.reset_index(drop=True)

In [ ]:
#Delete the data with unreasonable size
# DBSCAN using size to rooms ratio
df_train['sqft_to_rooms'] = df_train['size_sqft'] / (df_train['num_beds'] + df_train['num_baths'])
sk_clustering_iris = DBSCAN(eps=50, min_samples=5).fit(df_train[['sqft_to_rooms']])
sk_noise_iris = np.argwhere(sk_clustering_iris.labels_ < 0).squeeze()
sk_noise_iris.sort()

for i in sk_noise_iris:
    df_train = df_train.drop(i)
    df_train = df_train.reset_index(drop=True)


# DBSCAN using rooms to size ratio
df_train['rooms_to_sqft'] = (df_train['num_beds'] + df_train['num_baths']) / df_train['size_sqft']
sk_clustering_iris = DBSCAN(eps=0.0005, min_samples=5).fit(df_train[['rooms_to_sqft']])
sk_noise_iris = np.argwhere(sk_clustering_iris.labels_ < 0).squeeze()
sk_noise_iris.sort()

for i in sk_noise_iris:
    df_train = df_train.drop(i)
    df_train = df_train.reset_index(drop=True)

df_train.drop(['rooms_to_sqft', 'sqft_to_rooms', 'baths_to_beds', 'beds_to_baths'], axis=1, inplace=True)

In [ ]:
# Use new feature price_per_sqft to detect unreasonable price
df_train["price_per_sqft"] = df_train["price"]/df_train["size_sqft"]

In [ ]:
import seaborn as sns
print(df_train["price_per_sqft"].describe())
sns.boxplot(y = df_train["price_per_sqft"])

In [ ]:
# Drop the unreasonable data based on 3-sigma rules
while True:
    mean = np.mean(df_train["price_per_sqft"])
    std = np.std(df_train["price_per_sqft"])
    high = mean + 3*std
    low = mean - 3*std
    if ((df_train["price_per_sqft"]>low).all() and (df_train["price_per_sqft"]<high).all()) == True:
        break
    else:
        df_train = df_train[df_train["price_per_sqft"] > low]
        df_train = df_train[df_train["price_per_sqft"] < high]

In [ ]:
print(df_train["price_per_sqft"].describe())
sns.boxplot(y = df_train["price_per_sqft"])

In [ ]:
# Find a boundary to remove the unreasonably small data
fig, ax =plt.subplots(1,3,constrained_layout=True, figsize=(12, 3))
s1=sns.distplot(df_train["price_per_sqft"], ax=ax[0])
s1.set_title("all data")
s2=sns.distplot(df_train[df_train["price_per_sqft"]<1000]["price_per_sqft"], ax=ax[1])
s2.set_title("<1000")
s3=sns.distplot(df_train[df_train["price_per_sqft"]<400]["price_per_sqft"], ax=ax[2])
s3.set_title("<400")

In [ ]:
# Unreasonable data still exists, and drop them.
df_train = df_train[df_train["price_per_sqft"] > 300]

In [ ]:
# Create new feature num_rooms
df_train["num_rooms"] = df_train["num_beds"] + df_train["num_baths"]

In [ ]:
df_train = df_train.reset_index(drop = True)

In [ ]:
df_hdb = df_train[df_train['property_type'] == 'hdb'].loc[:, ['lat', 'lng']]
df_hdb

In [ ]:
df_hdb = df_hdb.groupby(['lat', 'lng']).size().reset_index(name='counts')
fig = px.density_mapbox(df_hdb, lat='lat', lon='lng', z='counts',
                        mapbox_style="stamen-terrain")
 
fig

In [ ]:
df_condo = df_train[df_train['property_type'] == 'condo'].loc[:, ['lat', 'lng']]
df_condo

In [ ]:
df_condo = df_condo.groupby(['lat', 'lng']).size().reset_index(name='counts')
fig = px.density_mapbox(df_condo, lat='lat', lon='lng', z='counts',
                        mapbox_style="stamen-terrain")
 
fig

In [ ]:
df_train_hdb = df_train[df_train['property_type'] == 'hdb'].loc[:, ['lat', 'lng']]

# Normalize the dataset
for i in list(df_train_hdb.columns):
    df_train_hdb[i] = (df_train_hdb[i]-min(df_train_hdb[i]))/(max(df_train_hdb[i]) - min(df_train_hdb[i]))
    

SSE = []
for cluster in range(1,20):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(df_train_hdb)
    SSE.append(kmeans.inertia_)
    
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
kmeans = KMeans(n_clusters = 4, init='k-means++')
kmeans.fit(df_train_hdb)
labels = kmeans.labels_

df_train_hdb = df_train[df_train['property_type'] == 'hdb'].loc[:, ['lat', 'lng']]

df_train_hdb['labels'] = labels

df_train_hdb = df_train_hdb.drop_duplicates(subset=['lat', 'lng'])

fig = px.scatter_mapbox(df_train_hdb, lat='lat', lon='lng', color='labels',
                        mapbox_style="open-street-map")
 
fig

In [ ]:
df_train_condo = df_train[df_train['property_type'] == 'condo'].loc[:, ['lat', 'lng']]

# Normalize the dataset
for i in list(df_train_condo.columns):
    df_train_condo[i] = (df_train_condo[i]-min(df_train_condo[i]))/(max(df_train_condo[i]) - min(df_train_condo[i]))
    
SSE = []
for cluster in range(1,20):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(df_train_condo)
    SSE.append(kmeans.inertia_)
    
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
kmeans = KMeans(n_clusters = 4, init='k-means++')
kmeans.fit(df_train_condo)
labels = kmeans.labels_

df_train_condo = df_train[df_train['property_type'] == 'condo'].loc[:, ['lat', 'lng']]

df_train_condo['labels'] = labels

df_train_condo = df_train_condo.drop_duplicates(subset=['lat', 'lng'])

fig = px.scatter_mapbox(df_train_condo, lat='lat', lon='lng', color='labels',
                        mapbox_style="open-street-map")
 
fig

In [ ]:
import copy
train = copy.deepcopy(df_train)

In [ ]:
temp = train.groupby("property_type").mean()
temp = temp.sort_values(by="price",ascending=True)
ax_order = temp.index.tolist()
fig, ax =plt.subplots(1,2,constrained_layout=True, figsize=(12, 5))
s1 = sns.barplot(y="price", x="property_type", data=train, ax=ax[0])
s1.set_xticklabels(s1.get_xticklabels(),rotation = 80)
s2 = sns.barplot(y="price", x="property_type", data=train, order=ax_order, ax=ax[1])
s2.set_xticklabels(s2.get_xticklabels(),rotation = 80)

In [ ]:
# Encoding property_type based on the visualization above
train = train.replace(["hdb", "hdb executive", "walk-up", "executive condo", "shophouse"],[0,0,0,0,0])
train = train.replace(["condo", "apartment", "landed", "terraced house", "cluster house"],[1,1,1,1,1])
train = train.replace(["townhouse", "corner terrace", "good class bungalow", "semi-detached house"],[2, 2, 2, 2])
train = train.replace(["bungalow"], [3])

In [ ]:
# Encoding the tenure
train = train.replace(["99-year leasehold", "110-year leasehold", "103-year leasehold", "102-year leasehold", "100-year leasehold"],[0,0,0,0,0])
train = train.replace(["999-year leasehold", "946-year leasehold", "956-year leasehold", "929-year leasehold", "947-year leasehold"],[1,1,1,1,1])
train = train.replace(["freehold"],[2])

In [ ]:
# Encoding the built_year
#discrete
#0-1963-1970
#1-1971-1990
#2-1991-2005
#3-2006-2020
#4-2021-2028
for i in range(train.shape[0]):
    if train.iloc[i,6]<=1970:
        train.iloc[i,6]=0
    if train.iloc[i,6]<=1990 and train.iloc[i,6]>=1971:
        train.iloc[i,6]=1
    if train.iloc[i,6]>=1991 and train.iloc[i,6]<=2005:
        train.iloc[i,6]=2
    if train.iloc[i,6]>=2006 and train.iloc[i,6]<=2020:
        train.iloc[i,6]=3
    if train.iloc[i,6]>=2021:
        train.iloc[i,6]=4

In [ ]:
# Define a function to find the mininum distance, to deal with the auxilary data.
def min_dist(lat, lng, df):
    return min(np.sqrt((df["lat"]-lat)**2+(df["lng"]-lng)**2))

In [ ]:
commercial_centres = pd.read_csv(r"auxiliary-data\sg-commerical-centres.csv")
mrt_stations = pd.read_csv(r"auxiliary-data\sg-mrt-stations.csv")
primary_schools = pd.read_csv(r"auxiliary-data\sg-primary-schools.csv")
secondary_schools = pd.read_csv(r"auxiliary-data\sg-secondary-schools.csv")
shopping_malls = pd.read_csv(r"auxiliary-data\sg-shopping-malls.csv")
subzones = pd.read_csv(r"auxiliary-data\sg-subzones.csv")

In [ ]:
# Find the distance of nearest mrt station, primary school and shopping mall.
min_dist_commercial = []
min_dist_mrt = []
min_dist_pri = []
min_dist_sec = []
min_dist_mall = []
for i in range(train.shape[0]):
    min_dist_commercial.append(min_dist(train["lat"][i],train["lng"][i],commercial_centres))
    min_dist_mrt.append(min_dist(train["lat"][i],train["lng"][i],mrt_stations))
    min_dist_pri.append(min_dist(train["lat"][i],train["lng"][i],primary_schools))
    min_dist_sec.append(min_dist(train["lat"][i],train["lng"][i],secondary_schools))
    min_dist_mall.append(min_dist(train["lat"][i],train["lng"][i],shopping_malls))

In [ ]:
# Normalize the data
train["dist_mrt"] = (np.array(min_dist_mrt)-min(min_dist_mrt))/(max(min_dist_mrt)-min(min_dist_mrt))
train["dist_pri"] = (np.array(min_dist_pri)-min(min_dist_pri))/(max(min_dist_pri)-min(min_dist_pri))
train["dist_mall"] = (np.array(min_dist_mall)-min(min_dist_mall))/(max(min_dist_mall)-min(min_dist_mall))
train["dist_commercial"] = (np.array(min_dist_commercial)-min(min_dist_commercial))/(max(min_dist_commercial)-min(min_dist_commercial))
train["dist_sec"] = (np.array(min_dist_sec)-min(min_dist_sec))/(max(min_dist_sec)-min(min_dist_sec))

In [ ]:
temp = train.groupby("subzone").mean()
temp = temp.sort_values(by="price",ascending=True)
ax_order = temp.index.tolist()
fig, ax =plt.subplots(2, 1,constrained_layout=True, figsize=(60, 40))
s1 = sns.barplot(y="price", x="subzone", data=train, ax=ax[0])
s1.set_xticklabels(s1.get_xticklabels(),rotation = 80)
s2 = sns.barplot(y="price", x="subzone", data=train, order=ax_order, ax=ax[1])
s2.set_xticklabels(s2.get_xticklabels(),rotation = 80)
plt.subplot(2, 1, 1)
plt.xlabel("Subzone", fontsize = 30)
plt.ylabel("Average Price", fontsize = 30)
plt.subplot(2, 1, 2)
plt.xlabel("Subzone", fontsize = 30)
plt.ylabel("Average Price", fontsize = 30)

In [ ]:
# Use the average price of all the houses in a subzone to encode the feature subzone
for i in list(set(train["subzone"])):
    temp = train[train["subzone"] == i]
    train = train.replace(i, np.mean(temp["price"]))

In [ ]:
train.columns

In [ ]:
train_hdb = train[train['property_type'] == 0]

train_hdb.columns

In [ ]:
train_hdb = train_hdb.drop(columns=['listing_id', 'title', 'address', 'property_name', 'property_type', 'num_beds', 'num_baths', 'floor_level', 'furnishing', 'available_unit_types', 
                        'total_num_units', 'property_details_url', 'lat', 'lng', 'elevation','planning_area', 'lease_end_year', 'tenure'])

train_hdb

In [ ]:
# Normalize the dataset
for i in range(6, 11):
    train_hdb.iloc[:, i] = 1 / train_hdb.iloc[:, i]

train_hdb

In [ ]:
# Normalize the dataset
for i in list(train_hdb.columns):
    train_hdb[i] = (train_hdb[i]-min(train_hdb[i]))/(max(train_hdb[i]) - min(train_hdb[i]))

In [ ]:
train_hdb.to_csv('train_hdb.csv', index=False)

In [ ]:
SSE = []
for cluster in range(1,20):
    kmeans = KMeans(n_clusters = cluster, init='k-means++')
    kmeans.fit(train_hdb)
    SSE.append(kmeans.inertia_)
    
frame = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE})
plt.figure(figsize=(12,6))
plt.plot(frame['Cluster'], frame['SSE'], marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')

In [ ]:
kmeans = KMeans(n_clusters = 5, init='k-means++')
kmeans.fit(train_hdb)
labels = kmeans.labels_

In [ ]:
cluster = train_hdb

cluster['labels'] = labels

cluster = cluster.groupby(by=['labels']).mean()

In [ ]:
# Normalize the dataset
for i in range(0, 11):
    cluster.iloc[:, i] = (cluster.iloc[:, i] - cluster.iloc[:, i].min()) / (cluster.iloc[:, i].max() - cluster.iloc[:, i].min())

In [ ]:
cluster

In [ ]:
categories = ['built_year', 'size_sqft', 'subzone', 'price', 'price_per_sqft', 'num_rooms', 'dist_mrt', 'dist_pri', 'dist_mall', 'dist_commercial', 'dist_sec']
fig = go.Figure()

for index, row in cluster.iterrows():
    if index == 4:
        fig.add_trace(go.Scatterpolar(
            r = [row['built_year'], row['size_sqft'], row['subzone'], row['price'], row['price_per_sqft'], row['num_rooms'], row['dist_mrt'], row['dist_pri'], row['dist_mall'], row['dist_commercial'], row['dist_sec']],
          theta=categories,
          fill='toself',
          name='Cluster {}'.format(index)
        ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      # range=[0, 5]
    )),
  showlegend=False
)

fig.show()